In [20]:
%reload_ext autoreload
%autoreload 2

from importlib import import_module

from src.deep_image_matching import logger, timer
from src.deep_image_matching.config import Config
from src.deep_image_matching.image_matching import ImageMatching
from src.deep_image_matching.io.h5_to_db import export_to_colmap
from src.deep_image_matching.parser import parse_config

Get the list of possible configurations

In [27]:
Config.get_config_names()

['superpoint+lightglue',
 'superpoint+lightglue_fast',
 'superpoint+superglue',
 'disk+lightglue',
 'aliked+lightglue',
 'orb+kornia_matcher',
 'sift+kornia_matcher',
 'loftr',
 'roma',
 'keynetaffnethardnet+kornia_matcher']

build the configuration dictionary

In [25]:
cfg = Config.from_name("superpoint+lightglue")
cfg

{'extractor': {'name': 'superpoint',
  'keypoint_threshold': 0.0001,
  'max_keypoints': 4096},
 'matcher': {'name': 'lightglue',
  'n_layers': 9,
  'flash': True,
  'depth_confidence': -1,
  'width_confidence': -1,
  'filter_threshold': 0.5},
 'general': {'quality': <Quality.HIGH: 3>,
  'tile_selection': <TileSelection.PRESELECTION: 3>,
  'tiling_grid': [3, 3],
  'tiling_overlap': 0,
  'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
  'gv_threshold': 4,
  'gv_confidence': 0.9999,
  'preselection_size_max': 2000}}

In [3]:



cfg_general_user = {
    "image_dir": args.images,
    "output_dir": args.outs,
    "matching_strategy": args.strategy,
    "retrieval": args.retrieval,
    "pair_file": args.pairs,
    "overlap": args.overlap,
    "upright": args.upright,
    "verbose": args.verbose,
    "skip_reconstruction": args.skip_reconstruction,
}


imgs_dir = config["general"]["image_dir"]
output_dir = config["general"]["output_dir"]
matching_strategy = config["general"]["matching_strategy"]
retrieval_option = config["general"]["retrieval"]
pair_file = config["general"]["pair_file"]
overlap = config["general"]["overlap"]
upright = config["general"]["upright"]
extractor = config["extractor"]["name"]
matcher = config["matcher"]["name"]

usage: ipykernel_launcher.py [-h] [--gui] [-i IMAGES] [-o OUTS]
                             [-c {superpoint+lightglue,superpoint+lightglue_fast,superpoint+superglue,disk+lightglue,aliked+lightglue,orb+kornia_matcher,sift+kornia_matcher,loftr,roma,keynetaffnethardnet+kornia_matcher}]
                             [-m {bruteforce,sequential,retrieval,custom_pairs,matching_lowres}]
                             [-p PAIRS] [-v OVERLAP]
                             [-r {netvlad,openibl,cosplace,dir}] [--upright]
                             [--skip_reconstruction] [-f] [-V]
ipykernel_launcher.py: error: argument -f/--force: ignored explicit argument '/home/francesco/.local/share/jupyter/runtime/kernel-v2-1069879Iy28PkzB563i.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
# Initialize ImageMatching class
img_matching = ImageMatching(
    imgs_dir=imgs_dir,
    output_dir=output_dir,
    matching_strategy=matching_strategy,
    retrieval_option=retrieval_option,
    local_features=extractor,
    matching_method=matcher,
    pair_file=pair_file,
    custom_config=config,
    overlap=overlap,
)